<a href="https://colab.research.google.com/github/DeokwonWang/KoBERTtest/blob/main/KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==4.8.1
!pip install torch
!pip install folium==0.2.1

     |████████████████████████████████| 47.3 MB 1.9 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=f12dabb992e41cdeb26925b2695a0ab69411233904f560eb089588cee8019681
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 2.5 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 31.5 MB/s 
     |████████████████████████████████| 3.3 MB 32.8 MB/s 
     |████████████████████████████████| 69 kB 3.1 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=5f21a24ed8a2e8685f679bdfd2929f15d30d405c6a9ae58c5

In [ ]:
#KoBERT 파일 로드 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ax1gh6_y
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ax1gh6_y
     |████████████████████████████████| 132 kB 3.7 MB/s 
     |████████████████████████████████| 4.5 MB 36.2 MB/s 
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 8.5 MB 29.8 MB/s 
     |████████████████████████████████| 127 kB 51.9 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=0b6e83ef61779623f6daf8deb141420abf7d381b1be0ebfd9d7dadbe9b4c7793
  Stored in directory: /tmp/pip-ephem-wheel-cache-vkx1exch/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/챗봇/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
len(chatbot_data)

38594

In [ ]:
chatbot_data.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
14645,이런 개서리는 좀 하지맙시다.,분노,NaN,NaN,NaN,NaN,NaN
33084,맞는거가태 누군지는말못하는데 이름들으면다아는연옌이랑지금사귄다ㅋㅋㅋㅋㅋㅋㅋㅋ,행복,NaN,NaN,NaN,NaN,NaN
12896,죽을죄를 지었으면 그냥 죽지 그러냐,분노,NaN,NaN,NaN,NaN,NaN
29217,연습하느라 넘어지고 힘들었을텐데 너무 멋지게 경기에 임해주셔서 보는사람에게 감...,행복,NaN,NaN,NaN,NaN,NaN
6183,의외로 여성들이 꼽은 소원중 1위가 딴게아니고 먹어도 살안찌는거라더라,놀람,NaN,NaN,NaN,NaN,NaN
30588,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 중국한테 간신히 이겨서 암걸렸는데 이 기사보고 암이 나...,행복,NaN,NaN,NaN,NaN,NaN
9056,ㄹ혜 영화 가 더 재밋어서 충격오지도 않음,놀람,NaN,NaN,NaN,NaN,NaN
3809,그 날도 만나는 남자가 있는걸 알긴했지만 ?휴 너무복잡하네요,공포,NaN,NaN,NaN,NaN,NaN
30393,국내부자들은 한두푼 더 쳐먹을려고 계획중인데 빌게이츠는 저런 계획을 세우고있었다니 ...,행복,NaN,NaN,NaN,NaN,NaN
14504,나오지도 않고 위증도하고. . . .,분노,NaN,NaN,NaN,NaN,NaN


In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

30875
7719


In [ ]:
# BERT 모델에 들어가는 dataset 만들기
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return(self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return(len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 1171, 7095,  835, 6079, 1409, 5400, 5886, 5655,  940,  633,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(12, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 4)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 7,
               dr_rate = None,
               params = None) :
      super(BERTClassifier, self).__init__()
      self.bert = bert
      self.dr_rate = dr_rate

      self.classifier = nn.Linear(hidden_size, num_classes)
      if dr_rate:
        self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
      attention_mask = torch.zeros_like(token_ids)
      for i,v in enumerate(valid_length):
        attention_mask[i][:v] = 1
      return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
      attention_mask = self.gen_attention_mask(token_ids, valid_length)

      _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
      if self.dr_rate:
        out = self.dropout(pooler)
      return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# 옵티마이저와 스케쥴 준비(linear warmup and decay)
no_decay = ['bias', "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X,1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0
  model.train()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    loss = loss_fn(out, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step() # 학습률 업데이트 스케쥴
    train_acc += calc_accuracy(out, label)
    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
  print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
  print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/483 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.0124988555908203 train acc 0.125
epoch 1 batch id 201 loss 1.4362753629684448 train acc 0.26282649253731344
epoch 1 batch id 401 loss 1.2732006311416626 train acc 0.364908042394015
epoch 1 train acc 0.3893046449658768


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.5548116656071201


  0%|          | 0/483 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.7035439014434814 train acc 0.78125
epoch 5 batch id 201 loss 0.6421584486961365 train acc 0.7887126865671642
epoch 5 batch id 401 loss 0.4084732234477997 train acc 0.7972256857855362
epoch 5 train acc 0.8002238133578713


  0%|          | 0/121 [00:00<?, ?it/s]

epoch 5 test acc 0.5538150296673024


In [ ]:
torch.save(model, '/content/drive/MyDrive/챗봇/KoBERT_epoch5.pt')

In [ ]:
model1 = torch.load('/content/drive/MyDrive/챗봇/KoBERT_epoch5.pt')

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
def predict(predict_sentence):

  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers = 2)

  model1.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model1(token_ids, valid_length, segment_ids)


      test_eval=[]
      for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()

          if np.argmax(logits) == 0:
              test_eval.append("공포가")
          elif np.argmax(logits) == 1:
              test_eval.append("놀람이")
          elif np.argmax(logits) == 2:
              test_eval.append("분노가")
          elif np.argmax(logits) == 3:
              test_eval.append("슬픔이")
          elif np.argmax(logits) == 4:
              test_eval.append("중립이")
          elif np.argmax(logits) == 5:
              test_eval.append("행복이")
          elif np.argmax(logits) == 6:
              test_eval.append("혐오가")

      print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 접근금지 가처분 신청
>> 입력하신 내용에서 공포가 느껴집니다.


하고싶은 말을 입력해주세요 : 0


KAKAO speach-to-text 이용

In [ ]:
!pip install sounddevice
!pip install soundfile
!pip install scipy

ERROR: Could not find a version that satisfies the requirement queue (from versions: none)
ERROR: No matching distribution found for queue
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [ ]:
!pip install libportaudio2

ERROR: Could not find a version that satisfies the requirement libportaudio2 (from versions: none)
ERROR: No matching distribution found for libportaudio2


In [ ]:
import queue, os, threading
import sounddevice as sd
import soundfile as sf
from scipy.io.wavfile import write

In [ ]:
q = queue.Queue()

recorder = False
recording = False

file_num = 0

In [ ]:
def complicated_save(indata, frames, time, status):
	q.put(indata.copy())

def complicated_record():
  global file_num

  with sf.SoundFile("/content/drive/MyDrive/챗봇/음성녹음데이터/음성녹음데이터{0}.wav".format(file_num), mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
      with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save): 
          while recording: 
            file.write(q.get())
       
def start():
  global recorder
  global recording

  recording = True
  recorder = threading.Thread(target=complicated_record)
  print('start recording')
  recorder.start()

def stop():
  global recorder
  global recording
  global file_num

  recording = False
  recorder.join()

  print('stop recording')
  file_num += 1

In [ ]:
import time

In [ ]:
start()
time.sleep(3)
stop

start recording


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-62-8ed9e82080fe>", line 8, in complicated_record
    with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
  File "/usr/local/lib/python3.7/dist-packages/sounddevice.py", line 1416, in __init__
    **_remove_self(locals()))
  File "/usr/local/lib/python3.7/dist-packages/sounddevice.py", line 812, in __init__
    extra_settings, samplerate)
  File "/usr/local/lib/python3.7/dist-packages/sounddevice.py", line 2654, in _get_stream_parameters
    info = query_devices(device)
  File "/usr/local/lib/python3.7/dist-packages/sounddevice.py", line 564, in query_devices
    raise PortAudioError('Error querying device {}'.format(device))
sounddevice.PortAudioError: Error querying de

<function __main__.stop>